In [1]:
import os 

In [2]:
os.chdir("C:/Users/User/Desktop/3-IDSD/mlops/MLOPS/MlopsProject")

In [3]:
%pwd

'C:\\Users\\User\\Desktop\\3-IDSD\\mlops\\MLOPS\\MlopsProject'

In [4]:
import mlflow

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/hannamhiri/MlopsProject.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="hannamhiri"
os.environ["MLFLOW_TRACKING_PASSWORD"]="d818c76624661ed3e44ed5cd15bb08d17cd94c4d"

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_dir: Path              # ⬅️ dossier contenant plusieurs modèles
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [7]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params              # ✅ TOUS les paramètres
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_dir=config.model_dir,     # ✅ dossier des modèles
            all_params=params,              # ✅ dict complet
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/hannamhiri/MlopsProject.mlflow",
        )

        return model_evaluation_config



In [13]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
from mlProject import logger


In [14]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred, prob=None):
        """
        actual : vraies valeurs
        pred : prédictions binaires
        prob : probabilité pour calcul ROC-AUC (optionnel)
        """
        accuracy = accuracy_score(actual, pred)
        precision = precision_score(actual, pred, zero_division=0)
        recall = recall_score(actual, pred, zero_division=0)
        f1 = f1_score(actual, pred, zero_division=0)
        roc_auc = roc_auc_score(actual, prob) if prob is not None else None

        return accuracy, precision, recall, f1, roc_auc

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        for model_file in os.listdir(self.config.model_dir):
            if not model_file.endswith(".pkl"):
                continue

            model_name = model_file.replace(".pkl", "")
            model_path = os.path.join(self.config.model_dir, model_file)

            logger.info(f"Evaluating model: {model_name}")

            model = joblib.load(model_path)

            with mlflow.start_run(run_name=model_name):
                # Prédictions binaires et probabilités si disponibles
                predictions = model.predict(test_x)
                probas = model.predict_proba(test_x)[:,1] if hasattr(model, "predict_proba") else None

                accuracy, precision, recall, f1, roc_auc = self.eval_metrics(test_y, predictions, probas)

                scores = {
                    "accuracy": accuracy,
                    "precision": precision,
                    "recall": recall,
                    "f1": f1,
                    "roc_auc": roc_auc
                }

                save_json(
                    path=Path(os.path.join(self.config.root_dir, f"{model_name}_metrics.json")),
                    data=scores
                )

                # 🔹 Log params du modèle si existant
                if hasattr(self.config, "all_params") and model_name in self.config.all_params:
                    mlflow.log_params(self.config.all_params[model_name])

                # 🔹 Log metrics
                mlflow.log_metric("accuracy", accuracy)
                mlflow.log_metric("precision", precision)
                mlflow.log_metric("recall", recall)
                mlflow.log_metric("f1", f1)
                if roc_auc is not None:
                    mlflow.log_metric("roc_auc", roc_auc)

                # 🔹 Log modèle
                if tracking_url_type_store != "file":
                    mlflow.sklearn.log_model(
                        model,
                        "model",
                        registered_model_name=f"{model_name}Model"
                    )
                else:
                    mlflow.sklearn.log_model(model, "model")


In [15]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e



[2025-12-24 20:26:17,974: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-12-24 20:26:17,976: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-24 20:26:17,979: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-12-24 20:26:17,980: INFO: common: created directory at: artifacts]
[2025-12-24 20:26:17,981: INFO: common: created directory at: artifacts/model_evaluation]
[2025-12-24 20:26:18,008: INFO: 3320842624: Evaluating model: CatBoost]
[2025-12-24 20:26:18,292: INFO: common: json file saved at: artifacts\model_evaluation\CatBoost_metrics.json]


c:\Users\User\Desktop\3-IDSD\mlops\MLOPS\MlopsProject\base\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'CatBoostModel' already exists. Creating a new version of this model...
2025/12/24 20:26:27 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: CatBoostModel, version 6
Created version '6' of model 'CatBoostModel'.


[2025-12-24 20:26:27,568: INFO: 3320842624: Evaluating model: LightGBM]
[2025-12-24 20:26:29,297: INFO: common: json file saved at: artifacts\model_evaluation\LightGBM_metrics.json]


Registered model 'LightGBMModel' already exists. Creating a new version of this model...
2025/12/24 20:26:42 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: LightGBMModel, version 6
Created version '6' of model 'LightGBMModel'.


[2025-12-24 20:26:43,233: INFO: 3320842624: Evaluating model: RandomForest]
[2025-12-24 20:26:44,451: INFO: common: json file saved at: artifacts\model_evaluation\RandomForest_metrics.json]


Registered model 'RandomForestModel' already exists. Creating a new version of this model...
2025/12/24 20:31:22 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: RandomForestModel, version 6
Created version '6' of model 'RandomForestModel'.


[2025-12-24 20:31:22,815: INFO: 3320842624: Evaluating model: XGBoost]
[2025-12-24 20:31:23,829: INFO: common: json file saved at: artifacts\model_evaluation\XGBoost_metrics.json]


Registered model 'XGBoostModel' already exists. Creating a new version of this model...
2025/12/24 20:31:33 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: XGBoostModel, version 5
Created version '5' of model 'XGBoostModel'.


In [16]:
import mlflow
print("MLflow tracking URI:", mlflow.get_tracking_uri())


MLflow tracking URI: https://dagshub.com/hannamhiri/MlopsProject.mlflow
